In [68]:
import pandas as pd
import re
import json
import psycopg2
# import pandas
# import metadata_funs

In [69]:
ers_path = 'FY2014-19 Datasets for ERS Publications.xlsx'
ers_df_list = []
ers_xls = pd.ExcelFile(ers_path)
sheet_names = ers_xls.sheet_names

In [70]:
sheet_names_lim = ['Census Surveys', 'CES', 'CPS-FSS', 'Cost Est. Foodborne Illnesses'
                   , 'FDA-FSIS-AMS', 'FADS-LAFA', 'FARA', 'FICRCD-CSFII', 'FNS', 'FoodAPS'
                   ,'IRI', 'NHANES', 'Nielsen', 'Qtrly FAFH', 'SNAP Admin', 'TD Linx', 'SCH MEALS DATA']

In [80]:
def db_connect():
    conn = psycopg2.connect("dbname='rcdb' user='postgres' host='richcontext.cz5u3bpfaar4.us-east-1.rds.amazonaws.com' password='rc_dumps'")
    return conn

def gen_insert_vals(ers_pubs):
    Row_list =[] 
    for index, row in ers_pubs.iterrows(): 
        my_list =[row.sheet_name,row.Author,row.Released,row.Series,row.Titles,row.Datasets]
        my_list = [str(m).strip() for m in my_list]
#         my_list = [re.sub('"',"'",m) for m in my_list]
#         my_list = [m.replace('\\"', "'") for m in my_list]
        my_list = [re.sub("'","''",m) for m in my_list]
        Row_list.append(my_list) 
    return Row_list

def insert_ers_linkages(Row_list):
    for r in Row_list:
        insert_statement = """INSERT INTO public.usda_linkages_20190913 (sheet_name, author, released, series, titles, datasets)
        VALUES {};""".format(tuple(r))
        conn = db_connect()
        cursor = conn.cursor()
        try:
            cursor.execute(insert_statement)
            conn.commit()
        except:
            pass
        
def run_select_query(conn,query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()
    return rows

In [72]:
def fetch_existing_records():
    conn = db_connect()
    select_query = """SELECT author, released, series, titles, datasets
    FROM public.usda_linkages_20190913; """
    existing_records = run_select_query(conn,select_query)  
    existing_records_list = [list(r) for r in existing_records]
    return existing_records_list

In [76]:
a = row_list[3]
# [m.replace(r'\"', "sup") for m in a]
# re.sub("'","''",a[4])

# # '"' in a[4]
# # "'" + a[4]
# a[4].replace(r'"',"'")
a[4].replace(r'"',"'")

"WIC Households'' Food Purchases Using WIC Benefits and Paying Out of Pocket: A Case Study of Cold Cereal Purchases"

In [79]:
row_list

[['Nielsen',
  'Oliveira',
  'FY14',
  'EIB-119',
  'Trends in Infant Formula Rebate Contracts: Implications for the WIC Program',
  '▪2008 proprietary report prepared for ERS by the Nielsen Company'],
 ['Nielsen',
  'Rahkovsky',
  'FY15',
  'ERR-195',
  'Food Choices and Store Proximity',
  '▪2010 Nielsen Homescan Panel Survey'],
 ['Nielsen',
  'Buzby',
  'FY16',
  'EIB-155',
  'Updated Supermarket Shrink Estimates for Fresh Foods and Their Implications for ERS Loss-Adjusted Food Availability Data',
  '▪Nielsen Perishables Group'],
 ['Nielsen',
  'Carlson',
  'FY16',
  'ERR-209',
  'Changes in Retail Organic Price Premiums from 2004 to 2010',
  '▪Nielsen Homescan data']]

In [81]:
for n in sheet_names_lim:
    existing_records = fetch_existing_records()
    ers_pubs = pd.read_excel(ers_path, sheet_name = n,header = 1)
    ers_pubs['sheet_name'] = n
    row_list = gen_insert_vals(ers_pubs)
    push_list = [p for p in row_list if p not in existing_records]
    if len(push_list) > 0:
        insert_ers_linkages(push_list)
    elif len(push_list) == 0:
        pass